### Jul AdventKalender D11

https://adventofcode.com/2022/day/11

In [1]:
import numpy as np

#### Day 11.1  

There is a group of monkeys playing with your items! Monkeys operate based on how worried you are about each item (The full example is in /data/input11_test.txt):

    Monkey 0:
      Starting items: 79, 98
      Operation: new = old * 19
      Test: divisible by 23
        If true: throw to monkey 2
        If false: throw to monkey 3

    Monkey 1:
    ......

1. Based on above monkey configuration, each monkey has several attributes:
    * **Starting items** lists your worry level for each item the monkey is currently holding in the order they will be inspected.
    * **Operation** shows how your worry level changes as that monkey inspects an item.
    * **Test** shows how the monkey uses your worry level to decide where to throw an item next.
        * If **true** shows what happens with an item if the Test was true.
        * If **false** shows what happens with an item if the Test was false.

2. Default worry level change:
    * After inspection (**Operation**) and before **Test**, your worry level is **divided by 3** and **rounded down** to the nearest integer because you are relived the item is not damaged.

For each round:

* The monkey inspects and throws the Starting items one at a time by order. (1&2)
* When a monkey throws an item to another monkey, the item goes on the **end** of the recipient monkey's Starting items list. 
* The turn of each monkey ends when their Starting items list is empty.
* For each round: Monkey 0 goes first, then monkey 1, and so on until each monkey has had one turn.

In the above example, the first round proceeds as follows (the full process can be found [Here](https://adventofcode.com/2022/day/11):

    Monkey 0:
      Monkey inspects an item with a worry level of 79.
        Worry level is multiplied by 19 to 1501.
        Monkey gets bored with item. Worry level is divided by 3 to 500.
        Current worry level is not divisible by 23.
        Item with worry level 500 is thrown to monkey 3.
      Monkey inspects an item with a worry level of 98.
        Worry level is multiplied by 19 to 1862.
        Monkey gets bored with item. Worry level is divided by 3 to 620.
        Current worry level is not divisible by 23.
        Item with worry level 620 is thrown to monkey 3.
    ......
    
Count the total number of times each monkey inspects items over **20** rounds, and choose **two** most active monkeys. The level of monkey business is **multiplying** the two together (10605). 

    Monkey 0 inspected items 101 times.*
    Monkey 1 inspected items 95 times.
    Monkey 2 inspected items 7 times.
    Monkey 3 inspected items 105 times.*

What is the level of monkey business after 20 rounds of stuff-slinging simian shenanigans, based on the full input data file?

In [2]:
def readMonkeyStrategies(file_name):
    lines_monkeys = {}
    f = open(file_name, "r")
    current_key = 0
    while True:
        line = f.readline()
        if not line:
            break
        if line.startswith('Monkey'):
            current_key = line.split(' ')[1][0]
            lines_monkeys[current_key]=[]
        elif len(line.strip())>0:
            lines_monkeys[current_key].append(line.strip())
    f.close()
    return lines_monkeys
def parseMonkeyAttributes(strategy):
    attr = {}
    for line in strategy:
        com = [i.strip() for i in line.split(':')]
        if com[0] == 'Starting items':
            attr['Starting items'] = list(map(int, com[1].split(',')))
        elif com[0] == 'Operation':
            attr['Operation'] = com[1].split(' ')[3:5]
        elif com[0] == 'Test':
            attr['Test'] = [int(com[1].split(' ')[2]), '', '']
        elif com[0] == 'If true':
            attr['Test'][1] = com[1].split(' ')[3]
        elif com[0] == 'If false':
            attr['Test'][2] = com[1].split(' ')[3]
    return attr
lines_monkeys = readMonkeyStrategies("data/input11.txt")

# to show what lines_monkeys is like
{k: lines_monkeys[k] for k in sorted(lines_monkeys.keys())[:2]}

{'0': ['Starting items: 78, 53, 89, 51, 52, 59, 58, 85',
  'Operation: new = old * 3',
  'Test: divisible by 5',
  'If true: throw to monkey 2',
  'If false: throw to monkey 7'],
 '1': ['Starting items: 64',
  'Operation: new = old + 7',
  'Test: divisible by 2',
  'If true: throw to monkey 3',
  'If false: throw to monkey 6']}

In [3]:
import operator
ops = { "+": operator.add, "-": operator.sub, "*": operator.mul , "/": operator.truediv }

def _operation(op, value_str, old):
    value = old if value_str == 'old' else int(value_str)
    return ops[op](old, value)

def excuteMonkeyRound(monkey_name):
    global attrs_monkeys
    items = attrs_monkeys[monkey_name]['Starting items']
    if len(items) == 0:
        return
    ops = attrs_monkeys[monkey_name]['Operation']
    test = attrs_monkeys[monkey_name]['Test']
    if 'Inspect times' not in attrs_monkeys[monkey_name]:
        attrs_monkeys[monkey_name]['Inspect times'] = 0
    for old in items:
        new = _operation(ops[0], ops[1], old)
        new = new//3
        monkey_next = test[1] if new%test[0] == 0 else test[2]
        attrs_monkeys[monkey_next]['Starting items'].append(new)
        attrs_monkeys[monkey_name]['Inspect times'] += 1
    attrs_monkeys[monkey_name]['Starting items'].clear()
#     for monkey_name in attrs_monkeys.keys():
#         print('Monkey {0} start items {1}.'.format(monkey_name, attrs_monkeys[monkey_name]['Starting items']))

def monkeyBusiness(top_num=2):
    return np.prod(np.sort([attrs_monkeys[monkey_name]['Inspect times'] for monkey_name in attrs_monkeys])[::-1][:top_num])

In [4]:
attrs_monkeys = {k: parseMonkeyAttributes(v) for k, v in lines_monkeys.items()}

# to show what attrs_monkeys is like
{k: attrs_monkeys[k] for k in sorted(attrs_monkeys.keys())[:2]}

{'0': {'Starting items': [78, 53, 89, 51, 52, 59, 58, 85],
  'Operation': ['*', '3'],
  'Test': [5, '2', '7']},
 '1': {'Starting items': [64], 'Operation': ['+', '7'], 'Test': [2, '3', '6']}}

In [5]:
iters = 20
for i in range(iters):
    for monkey_name in attrs_monkeys.keys():
        excuteMonkeyRound(monkey_name)

In [6]:
for monkey_name in attrs_monkeys.keys():
    print('Monkey {0} start items {1}.'.format(monkey_name, attrs_monkeys[monkey_name]['Inspect times']))
print('Money bussiness level:', monkeyBusiness())

Monkey 0 start items 228.
Monkey 1 start items 218.
Monkey 2 start items 20.
Monkey 3 start items 48.
Monkey 4 start items 55.
Monkey 5 start items 211.
Monkey 6 start items 188.
Monkey 7 start items 222.
Money bussiness level: 50616


#### Day 11.2

Now your worry levels are **no longer** divided by 3 after each item is inspected. Figure out **another way** to avoid overflow.

Starting again from the initial state in your puzzle input, what is the level of monkey business after **10000** rounds?

In [7]:
attrs_monkeys = {k: parseMonkeyAttributes(v) for k, v in lines_monkeys.items()}

# to show what attrs_monkeys is like
{k: attrs_monkeys[k] for k in sorted(attrs_monkeys.keys())[:2]}

{'0': {'Starting items': [78, 53, 89, 51, 52, 59, 58, 85],
  'Operation': ['*', '3'],
  'Test': [5, '2', '7']},
 '1': {'Starting items': [64], 'Operation': ['+', '7'], 'Test': [2, '3', '6']}}

In [8]:
# collect all items in item_list (item index is item ID, value is initial worry value)
# update the worry values of items in attrs_monkeys to item ids
item_list = [] 
for monkey_name in attrs_monkeys.keys():
    items = attrs_monkeys[monkey_name]['Starting items']
    for i in range(len(items)):
        item_list.append(items[i]) # init item worry value
        attrs_monkeys[monkey_name]['Starting items'][i] = len(item_list)-1 # change starting items with item id

# each monkey keeps a list of items (worry value is unique to their divisible condition)
# worry value of each item will be updated with the final mod result 
# worry value = Operation(previous worry value) % divisible condition in Test
# the updating value now remains small (not overflow) and robust to mod operation
monkey_test_list = {} 
for monkey_name in attrs_monkeys.keys():
    monkey_test_list[monkey_name]=item_list.copy()

In [9]:
# to show what attrs_monkeys is like
{k: attrs_monkeys[k] for k in sorted(attrs_monkeys.keys())[:2]}

{'0': {'Starting items': [0, 1, 2, 3, 4, 5, 6, 7],
  'Operation': ['*', '3'],
  'Test': [5, '2', '7']},
 '1': {'Starting items': [8], 'Operation': ['+', '7'], 'Test': [2, '3', '6']}}

In [10]:
def _manageWorry(monkey_name, ops, item_id):
    global attrs_monkeys, monkey_test_list
    for monkey in monkey_test_list:
        mod = attrs_monkeys[monkey]['Test'][0]
        # do operations on all items
        monkey_test_list[monkey][item_id] = _operation(ops[0], ops[1], monkey_test_list[monkey][item_id])
        # update mod result (new worry value) for all monkeys
        monkey_test_list[monkey][item_id] %= mod
    return monkey_test_list[monkey_name][item_id]

def excuteMonkeyRound2(monkey_name):
    global attrs_monkeys
    items = attrs_monkeys[monkey_name]['Starting items']
    if len(items) == 0:
        return
    ops = attrs_monkeys[monkey_name]['Operation']
    test = attrs_monkeys[monkey_name]['Test']
    if 'Inspect times' not in attrs_monkeys[monkey_name]:
        attrs_monkeys[monkey_name]['Inspect times'] = 0
    for item_id in items:
        test_res = _manageWorry(monkey_name, ops, item_id)
        monkey_next = test[1] if test_res == 0 else test[2]
        attrs_monkeys[monkey_next]['Starting items'].append(item_id)
        attrs_monkeys[monkey_name]['Inspect times'] += 1
    attrs_monkeys[monkey_name]['Starting items'].clear()
#     for monkey_name in attrs_monkeys.keys():
#         print('Monkey {0} start items {1}.'.format(monkey_name, attrs_monkeys[monkey_name]['Starting items']))

In [11]:
iters = 10000
for i in range(iters):
    for monkey_name in attrs_monkeys.keys():
        excuteMonkeyRound2(monkey_name)
    if i in np.array([1, 20, 1000, 4000, 8000])-1:
        print('== After round {0} =='.format(i+1))
        for monkey_name in attrs_monkeys.keys():
            print('Monkey {0} inspect {1} times.'.format(monkey_name, attrs_monkeys[monkey_name]['Inspect times']))

== After round 1 ==
Monkey 0 inspect 8 times.
Monkey 1 inspect 1 times.
Monkey 2 inspect 5 times.
Monkey 3 inspect 6 times.
Monkey 4 inspect 7 times.
Monkey 5 inspect 8 times.
Monkey 6 inspect 6 times.
Monkey 7 inspect 11 times.
== After round 20 ==
Monkey 0 inspect 197 times.
Monkey 1 inspect 195 times.
Monkey 2 inspect 75 times.
Monkey 3 inspect 35 times.
Monkey 4 inspect 192 times.
Monkey 5 inspect 158 times.
Monkey 6 inspect 196 times.
Monkey 7 inspect 169 times.
== After round 1000 ==
Monkey 0 inspect 10617 times.
Monkey 1 inspect 9774 times.
Monkey 2 inspect 4848 times.
Monkey 3 inspect 1673 times.
Monkey 4 inspect 10613 times.
Monkey 5 inspect 6602 times.
Monkey 6 inspect 9776 times.
Monkey 7 inspect 8165 times.
== After round 4000 ==
Monkey 0 inspect 42528 times.
Monkey 1 inspect 39237 times.
Monkey 2 inspect 19581 times.
Monkey 3 inspect 6574 times.
Monkey 4 inspect 42523 times.
Monkey 5 inspect 26236 times.
Monkey 6 inspect 39239 times.
Monkey 7 inspect 32711 times.
== After 

In [12]:
for monkey_name in attrs_monkeys.keys():
    print('Monkey {0} start items {1}.'.format(monkey_name, attrs_monkeys[monkey_name]['Inspect times']))
print('Money bussiness level:', monkeyBusiness())

Monkey 0 start items 106346.
Monkey 1 start items 98133.
Monkey 2 start items 49031.
Monkey 3 start items 16402.
Monkey 4 start items 106342.
Monkey 5 start items 65512.
Monkey 6 start items 98135.
Monkey 7 start items 81799.
Money bussiness level: 11309046332
